In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
#print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score

import lightgbm as lgb

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

plt.style.use('seaborn')
sns.set(font_scale=1)

In [3]:
sampling_flg=0 #サンプリング有無をコントロール
submit_flg=1 #保存するかをコントロール（サンプリングしない時のみ）

SEED=12345
sample_num=10000
fold_num=5

#train関連
train_dir='../../01_input/train.csv'
train_drop_col=['ID_code', 'target']
train_label='target'

#test関連
test_dir='../../01_input/test.csv'
test_drop_col=['ID_code']

#結果ファイル関連　nameは自分の名前に変更する
train_preds_dir='../../03_predict_train/name_200_LightGBM_train.csv'
test_preds_dir='../../04_predict_test/name_200_LightGBM_submission.csv'
save_col_name='oof_xgb'

sample_submission_dir='../../01_input/sample_submission.csv'
submission_target_col_name='target'
submission_id_col_name='ID_code'

In [45]:
random_state = 42
np.random.seed(random_state)
#ロード
df_train=pd.read_csv(train_dir)
df_test=pd.read_csv(test_dir)

In [46]:
def augment(x,y,t=2):
    xs,xn = [],[]
    for i in range(t):
        mask = y>0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        print(ids)
        print(x1.shape[1])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
            
        xs.append(x1)
    for i in range(t//2):
        mask = y==0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xn.append(x1)

    xs = np.vstack(xs)
    xn = np.vstack(xn)
    ys = np.ones(xs.shape[0])
    yn = np.zeros(xn.shape[0])
    x = np.vstack([x,xs,xn])
    y = np.concatenate([y,ys,yn])
    return x,y

In [47]:
features = [col for col in df_train.columns if col not in ['target', 'ID_code']]
X_test = df_test[features].values
X_train, y_train = df_train[features], df_train['target']

In [48]:
X_t, y_t = augment(X_train.values, y_train.values)

[    0     1     2 ... 20095 20096 20097]
200
[    0     1     2 ... 20095 20096 20097]
200


In [49]:
X_t.shape

(420098, 200)

In [5]:
lgb_params = {
    "objective" : "binary",
    "metric" : "auc",
    "boosting": 'gbdt',
    "max_depth" : -1,
    "num_leaves" : 13,
    "learning_rate" : 0.0095,
    "bagging_freq": 5,
    "bagging_fraction" : 0.35,
    "feature_fraction" : 0.05,
    "min_data_in_leaf": 80,
    "min_sum_heassian_in_leaf": 10,
    "tree_learner": "serial",
    "boost_from_average": "false",
    #"lambda_l1" : 5,
    #"lambda_l2" : 5,
    "bagging_seed" : random_state,
    "verbosity" : 1,
    "seed": random_state
}

In [6]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
oof = df_train[['ID_code', 'target']]
oof['predict'] = 0
predictions = df_test[['ID_code']]
val_aucs = []
feature_importance_df = pd.DataFrame()

In [7]:
features = [col for col in df_train.columns if col not in ['target', 'ID_code']]
X_test = df_test[features].values

In [ ]:
for fold, (trn_idx, val_idx) in enumerate(skf.split(df_train, df_train['target'])):
    X_train, y_train = df_train.iloc[trn_idx][features], df_train.iloc[trn_idx]['target']
    X_valid, y_valid = df_train.iloc[val_idx][features], df_train.iloc[val_idx]['target']
    
    N = 5
    p_valid,yp = 0,0
    for i in range(N):
        X_t, y_t = augment(X_train.values, y_train.values)
        X_t = pd.DataFrame(X_t)
        X_t = X_t.add_prefix('var_')
    
        trn_data = lgb.Dataset(X_t, label=y_t)
        val_data = lgb.Dataset(X_valid, label=y_valid)
        evals_result = {}
        lgb_clf = lgb.train(lgb_params,
                        trn_data,
                        100000000,
                        valid_sets = [trn_data, val_data],
                        early_stopping_rounds=3000,
                        verbose_eval = 1000,
                        evals_result=evals_result
                       )
        p_valid += lgb_clf.predict(X_valid)
        yp += lgb_clf.predict(X_test)
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = lgb_clf.feature_importance()
    fold_importance_df["fold"] = fold + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    oof['predict'][val_idx] = p_valid/N
    val_score = roc_auc_score(y_valid, p_valid)
    val_aucs.append(val_score)
    
    print("AUC = {}".format(roc_auc_score(y_valid, p_valid)))
    print(confusion_matrix(y_valid, pd.DataFrame(np.round(p_valid))))
    
    predictions['fold{}'.format(fold+1)] = yp/N

Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.892676	valid_1's auc: 0.882046
[2000]	training's auc: 0.901226	valid_1's auc: 0.888619
[3000]	training's auc: 0.907331	valid_1's auc: 0.892997
[4000]	training's auc: 0.912058	valid_1's auc: 0.895849
[5000]	training's auc: 0.915699	valid_1's auc: 0.897754
[6000]	training's auc: 0.918711	valid_1's auc: 0.898934
[7000]	training's auc: 0.921254	valid_1's auc: 0.899571
[8000]	training's auc: 0.923599	valid_1's auc: 0.899916
[9000]	training's auc: 0.925776	valid_1's auc: 0.900171
[10000]	training's auc: 0.927879	valid_1's auc: 0.900254
[11000]	training's auc: 0.929918	valid_1's auc: 0.900323
[12000]	training's auc: 0.931934	valid_1's auc: 0.900283
[13000]	training's auc: 0.933868	valid_1's auc: 0.900211
[14000]	training's auc: 0.93578	valid_1's auc: 0.900176
Early stopping, best iteration is:
[11125]	training's auc: 0.93018	valid_1's auc: 0.900342
Training until validation scores don't improve for 3000 r

[13000]	training's auc: 0.934307	valid_1's auc: 0.901133
[14000]	training's auc: 0.936196	valid_1's auc: 0.901105
[15000]	training's auc: 0.938071	valid_1's auc: 0.901067
Early stopping, best iteration is:
[12259]	training's auc: 0.932895	valid_1's auc: 0.901239
Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.892418	valid_1's auc: 0.883897
[2000]	training's auc: 0.901034	valid_1's auc: 0.890303
[3000]	training's auc: 0.907138	valid_1's auc: 0.894638
[4000]	training's auc: 0.911816	valid_1's auc: 0.897546
[5000]	training's auc: 0.915516	valid_1's auc: 0.899253
[6000]	training's auc: 0.918454	valid_1's auc: 0.900214
[7000]	training's auc: 0.921019	valid_1's auc: 0.900736
[8000]	training's auc: 0.92337	valid_1's auc: 0.901173
[9000]	training's auc: 0.925541	valid_1's auc: 0.9013
[10000]	training's auc: 0.927628	valid_1's auc: 0.901424
[11000]	training's auc: 0.929649	valid_1's auc: 0.901474
[12000]	training's auc: 0.931654	valid_1's auc: 0.901475
[

In [ ]:
mean_auc = np.mean(val_aucs)
std_auc = np.std(val_aucs)
all_auc = roc_auc_score(oof['target'], oof['predict'])
print("Mean auc: %.9f, std: %.9f. All auc: %.9f." % (mean_auc, std_auc, all_auc))

print("AUC = {}".format(roc_auc_score(oof['target'], oof['predict'])))
print(confusion_matrix(oof['target'], pd.DataFrame(oof['predict']))))

In [ ]:
cols = (feature_importance_df[["feature", "importance"]]
        .groupby("feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:1000].index)
best_features = feature_importance_df.loc[feature_importance_df.feature.isin(cols)]

plt.figure(figsize=(14,26))
sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance",ascending=False))
plt.title('LightGBM Features (averaged over folds)')
plt.tight_layout()
plt.savefig('lgbm_importances.png')

In [ ]:
# submission
predictions['target'] = np.mean(predictions[[col for col in predictions.columns if col not in ['ID_code', 'target']]].values, axis=1)
predictions.to_csv('lgb_all_predictions.csv', index=None)
sub_df = pd.DataFrame({"ID_code":df_test["ID_code"].values})
sub_df["target"] = predictions['target']
sub_df.to_csv("lgb_submission.csv", index=False)
oof.to_csv('lgb_oof.csv', index=False)